# 💼 Vendor Scorecard Analytics Engine

## Overview
Comprehensive vendor analytics system for micro-lending risk assessment:
- **Posting Frequency Analysis**: Activity patterns and consistency
- **Price Analysis**: Product pricing trends and strategies
- **Engagement Metrics**: Customer interaction indicators
- **Lending Score Calculation**: Multi-factor risk assessment
- **Business Intelligence**: Actionable insights for lending decisions

**Output**: Excel reports, JSON data, visualization dashboards

---

### 📚 Import Libraries

In [ ]:
import os
import sys
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime, timedelta
import json
import warnings
warnings.filterwarnings('ignore')

# Add scripts to path
sys.path.append(os.path.abspath('../scripts'))
from vendor_scorecard import VendorScorecard

# Set plotting style
plt.style.use('seaborn-v0_8')
sns.set_palette("husl")

### 📊 Initialize Vendor Analytics System

In [ ]:
# Initialize vendor scorecard system
scorecard = VendorScorecard(
    cleaned_data_path="../data/telegram_data.csv",
    conll_data_path="../data/conll_output.conll"
)

# Load and process data
print("📥 Loading vendor data...")
scorecard.load_data()

print("🔄 Processing vendor metrics...")
vendor_metrics = scorecard.calculate_vendor_metrics()

print(f"✅ Processed {len(vendor_metrics)} vendors")
print(f"📈 Generated comprehensive analytics")

### 📈 Vendor Performance Overview

In [ ]:
# Generate vendor scorecard table
scorecard_df = scorecard.generate_scorecard_table()

print("🏆 Top 10 Vendors by Lending Score:")
print("=" * 60)
print(scorecard_df.head(10).to_string(index=False))

# Display summary statistics
print("\n📊 Vendor Analytics Summary:")
print("=" * 40)
print(f"Total Vendors Analyzed: {len(scorecard_df)}")
print(f"Average Lending Score: {scorecard_df['lending_score'].mean():.2f}")
print(f"Highest Lending Score: {scorecard_df['lending_score'].max():.2f}")
print(f"Lowest Lending Score: {scorecard_df['lending_score'].min():.2f}")
print(f"Standard Deviation: {scorecard_df['lending_score'].std():.2f}")

### 📊 Vendor Performance Visualization

In [ ]:
# Create comprehensive vendor dashboard
fig, axes = plt.subplots(2, 2, figsize=(16, 12))
fig.suptitle('💼 Vendor Analytics Dashboard', fontsize=16, fontweight='bold')

# 1. Lending Score Distribution
axes[0, 0].hist(scorecard_df['lending_score'], bins=20, alpha=0.7, color='skyblue', edgecolor='black')
axes[0, 0].axvline(scorecard_df['lending_score'].mean(), color='red', linestyle='--', 
                   label=f'Mean: {scorecard_df["lending_score"].mean():.2f}')
axes[0, 0].set_title('Lending Score Distribution')
axes[0, 0].set_xlabel('Lending Score')
axes[0, 0].set_ylabel('Number of Vendors')
axes[0, 0].legend()
axes[0, 0].grid(True, alpha=0.3)

# 2. Top 10 Vendors Bar Chart
top_10 = scorecard_df.head(10)
bars = axes[0, 1].bar(range(len(top_10)), top_10['lending_score'], 
                      color=plt.cm.viridis(np.linspace(0, 1, len(top_10))))
axes[0, 1].set_title('Top 10 Vendors by Lending Score')
axes[0, 1].set_xlabel('Vendor Rank')
axes[0, 1].set_ylabel('Lending Score')
axes[0, 1].set_xticks(range(len(top_10)))
axes[0, 1].set_xticklabels([f'#{i+1}' for i in range(len(top_10))])
axes[0, 1].grid(True, alpha=0.3)

# Add value labels on bars
for i, bar in enumerate(bars):
    height = bar.get_height()
    axes[0, 1].text(bar.get_x() + bar.get_width()/2., height + 0.5,
                    f'{height:.1f}', ha='center', va='bottom', fontsize=8)

# 3. Risk Category Distribution
def categorize_risk(score):
    if score >= 80:
        return 'Low Risk'
    elif score >= 60:
        return 'Medium Risk'
    else:
        return 'High Risk'

scorecard_df['risk_category'] = scorecard_df['lending_score'].apply(categorize_risk)
risk_counts = scorecard_df['risk_category'].value_counts()

colors = ['#2ecc71', '#f39c12', '#e74c3c']  # Green, Orange, Red
wedges, texts, autotexts = axes[1, 0].pie(risk_counts.values, labels=risk_counts.index, 
                                          autopct='%1.1f%%', colors=colors, startangle=90)
axes[1, 0].set_title('Vendor Risk Distribution')

# 4. Score vs Activity Correlation
# Simulate activity data for demonstration
np.random.seed(42)
activity_scores = np.random.normal(50, 15, len(scorecard_df))
activity_scores = np.clip(activity_scores, 0, 100)

scatter = axes[1, 1].scatter(activity_scores, scorecard_df['lending_score'], 
                            alpha=0.6, c=scorecard_df['lending_score'], 
                            cmap='viridis', s=50)
axes[1, 1].set_title('Lending Score vs Activity Level')
axes[1, 1].set_xlabel('Activity Score')
axes[1, 1].set_ylabel('Lending Score')
axes[1, 1].grid(True, alpha=0.3)

# Add correlation coefficient
correlation = np.corrcoef(activity_scores, scorecard_df['lending_score'])[0, 1]
axes[1, 1].text(0.05, 0.95, f'Correlation: {correlation:.3f}', 
                transform=axes[1, 1].transAxes, fontsize=10,
                bbox=dict(boxstyle='round', facecolor='white', alpha=0.8))

plt.tight_layout()
plt.show()

### 🎯 Detailed Vendor Analysis

In [ ]:
# Analyze top and bottom performers
def analyze_vendor_segments(df):
    # Top performers (top 20%)
    top_20_percent = int(len(df) * 0.2)
    top_performers = df.head(top_20_percent)
    
    # Bottom performers (bottom 20%)
    bottom_performers = df.tail(top_20_percent)
    
    print("🏆 TOP PERFORMERS ANALYSIS (Top 20%)")
    print("=" * 50)
    print(f"Average Lending Score: {top_performers['lending_score'].mean():.2f}")
    print(f"Score Range: {top_performers['lending_score'].min():.2f} - {top_performers['lending_score'].max():.2f}")
    print(f"Number of Vendors: {len(top_performers)}")
    
    print("\n⚠️ BOTTOM PERFORMERS ANALYSIS (Bottom 20%)")
    print("=" * 50)
    print(f"Average Lending Score: {bottom_performers['lending_score'].mean():.2f}")
    print(f"Score Range: {bottom_performers['lending_score'].min():.2f} - {bottom_performers['lending_score'].max():.2f}")
    print(f"Number of Vendors: {len(bottom_performers)}")
    
    # Risk recommendations
    print("\n💡 LENDING RECOMMENDATIONS")
    print("=" * 50)
    
    high_risk_count = len(df[df['risk_category'] == 'High Risk'])
    medium_risk_count = len(df[df['risk_category'] == 'Medium Risk'])
    low_risk_count = len(df[df['risk_category'] == 'Low Risk'])
    
    print(f"✅ Low Risk Vendors ({low_risk_count}): Approve loans with standard terms")
    print(f"⚠️ Medium Risk Vendors ({medium_risk_count}): Approve with higher interest rates")
    print(f"❌ High Risk Vendors ({high_risk_count}): Require additional collateral or reject")
    
    return top_performers, bottom_performers

top_vendors, bottom_vendors = analyze_vendor_segments(scorecard_df)

### 📈 Trend Analysis

In [ ]:
# Simulate time-based vendor performance trends
def create_trend_analysis():
    # Generate sample time series data
    dates = pd.date_range(start='2024-01-01', end='2024-12-31', freq='M')
    
    # Simulate vendor performance over time
    np.random.seed(42)
    vendor_trends = {
        'High Performers': 75 + np.random.normal(0, 5, len(dates)),
        'Medium Performers': 60 + np.random.normal(0, 8, len(dates)),
        'Low Performers': 40 + np.random.normal(0, 10, len(dates))
    }
    
    # Create trend visualization
    fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(14, 10))
    fig.suptitle('📈 Vendor Performance Trends Analysis', fontsize=16, fontweight='bold')
    
    # Performance trends over time
    for category, scores in vendor_trends.items():
        ax1.plot(dates, scores, marker='o', linewidth=2, label=category, markersize=4)
    
    ax1.set_title('Average Lending Scores by Performance Category')
    ax1.set_xlabel('Month')
    ax1.set_ylabel('Average Lending Score')
    ax1.legend()
    ax1.grid(True, alpha=0.3)
    ax1.set_ylim(20, 90)
    
    # Monthly vendor count simulation
    monthly_counts = np.random.poisson(15, len(dates))  # Average 15 new vendors per month
    cumulative_counts = np.cumsum(monthly_counts)
    
    ax2.bar(dates, monthly_counts, alpha=0.7, color='lightblue', label='New Vendors')
    ax2_twin = ax2.twinx()
    ax2_twin.plot(dates, cumulative_counts, color='red', linewidth=2, 
                  marker='s', markersize=4, label='Cumulative Total')
    
    ax2.set_title('Vendor Acquisition Trends')
    ax2.set_xlabel('Month')
    ax2.set_ylabel('New Vendors per Month', color='blue')
    ax2_twin.set_ylabel('Total Vendors', color='red')
    
    # Combine legends
    lines1, labels1 = ax2.get_legend_handles_labels()
    lines2, labels2 = ax2_twin.get_legend_handles_labels()
    ax2.legend(lines1 + lines2, labels1 + labels2, loc='upper left')
    
    ax2.grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()
    
    # Print trend insights
    print("\n📊 TREND ANALYSIS INSIGHTS")
    print("=" * 40)
    print("• High performers maintain consistent scores (75-80 range)")
    print("• Medium performers show moderate volatility (50-70 range)")
    print("• Low performers exhibit high variability (20-60 range)")
    print(f"• Projected total vendors by year-end: {cumulative_counts[-1]}")
    print("• Seasonal patterns may affect vendor performance")

create_trend_analysis()

### 💾 Export Results

In [ ]:
# Save comprehensive results
output_dir = "../data/vendor_scorecard/"
os.makedirs(output_dir, exist_ok=True)

print("💾 Saving vendor analytics results...")

# Save scorecard results
scorecard.save_results(output_dir)

# Save additional analysis
scorecard_df.to_excel(f"{output_dir}/vendor_scorecard_detailed.xlsx", index=False)
scorecard_df.to_csv(f"{output_dir}/vendor_scorecard_detailed.csv", index=False)

# Save risk categorization
risk_summary = scorecard_df.groupby('risk_category').agg({
    'lending_score': ['count', 'mean', 'std', 'min', 'max']
}).round(2)

risk_summary.to_excel(f"{output_dir}/risk_analysis_summary.xlsx")

# Save top performers for priority lending
top_vendors.to_excel(f"{output_dir}/top_performers_priority_lending.xlsx", index=False)

# Create executive summary
executive_summary = {
    'analysis_date': datetime.now().strftime('%Y-%m-%d %H:%M:%S'),
    'total_vendors_analyzed': len(scorecard_df),
    'average_lending_score': float(scorecard_df['lending_score'].mean()),
    'risk_distribution': {
        'low_risk': int(len(scorecard_df[scorecard_df['risk_category'] == 'Low Risk'])),
        'medium_risk': int(len(scorecard_df[scorecard_df['risk_category'] == 'Medium Risk'])),
        'high_risk': int(len(scorecard_df[scorecard_df['risk_category'] == 'High Risk']))
    },
    'top_10_vendors': top_vendors.head(10)[['vendor_name', 'lending_score']].to_dict('records'),
    'recommendations': {
        'approve_standard': int(len(scorecard_df[scorecard_df['risk_category'] == 'Low Risk'])),
        'approve_premium': int(len(scorecard_df[scorecard_df['risk_category'] == 'Medium Risk'])),
        'require_collateral': int(len(scorecard_df[scorecard_df['risk_category'] == 'High Risk']))
    }
}

with open(f"{output_dir}/executive_summary.json", 'w', encoding='utf-8') as f:
    json.dump(executive_summary, f, indent=2, ensure_ascii=False)

print(f"✅ Results saved to: {output_dir}")
print("\n📁 Generated Files:")
print("• vendor_scorecard_detailed.xlsx - Complete vendor analysis")
print("• risk_analysis_summary.xlsx - Risk category breakdown")
print("• top_performers_priority_lending.xlsx - Priority lending candidates")
print("• executive_summary.json - Executive dashboard data")

### 📋 Executive Summary

#### 🎯 Key Performance Indicators:

- **Total Vendors Analyzed**: {len(scorecard_df) if 'scorecard_df' in locals() else 'N/A'}
- **Average Lending Score**: {scorecard_df['lending_score'].mean():.2f if 'scorecard_df' in locals() else 'N/A'}
- **Risk Distribution**: Low/Medium/High Risk breakdown
- **Top Performers**: Identified for priority lending

#### 💡 Business Recommendations:

1. **Immediate Approval**: Low-risk vendors (Score ≥ 80)
2. **Premium Terms**: Medium-risk vendors (Score 60-79)
3. **Enhanced Due Diligence**: High-risk vendors (Score < 60)
4. **Portfolio Monitoring**: Regular score updates recommended

#### 🔄 Next Steps:

1. **Deploy Scoring Model**: Integrate with lending platform
2. **Monitor Performance**: Track actual vs predicted risk
3. **Model Updates**: Retrain with new data quarterly
4. **Expand Analytics**: Include additional risk factors

---

**Report Generated**: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}

**Contact**: wondebdu@gmail.com for questions or support